# The Assignment

In [1]:
# import the numpy and the matplotlib libraries
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# read the CSV file into the two-dimensional array "data"
# using numpy's function genfromtxt
data_set=np.genfromtxt('breast-cancer-wisconsin.data.csv',delimiter=',')
data_set = data_set.astype('int64')
print(data_set)

[[1000025       5       1 ...       1       1       2]
 [1002945       5       4 ...       2       1       2]
 [1015425       3       1 ...       1       1       2]
 ...
 [ 888820       5      10 ...      10       2       4]
 [ 897471       4       8 ...       6       1       4]
 [ 897471       4       8 ...       4       1       4]]


In [ ]:
# print the shape of the array "data"

print("Shape of data set array", data_set.shape)

In [ ]:
# print the first 5 rows of the array "data"
print("Printing first five rows of data set")
data_set[:5,:]

In [ ]:
# Assign to the array "X" the features from the array "data"
# do we need to ignore any feature (column)?

# Removing the unique patient id from features which is doesn't have any relation with output i.e data preprocessing 
X=np.copy(data_set[:,1:10]) 
print("Dataset after removing patient id")
print(X)

In [ ]:
# Assign to the array "y" the labels from the array "data"
# Do we need to adjust the values in "y"?

#Converting the actual output 2,4 to 0's and 1's

Y=np.copy(data_set[:,10:]) # Storing Expected Output in Y
y_conversion_list=[]
for i in Y:
    i=((i-2)/2)
    y_conversion_list.append(i)
Y=np.array(y_conversion_list)  # Actual output
y=np.array(Y)
y=y.astype(int)
y.shape


In [ ]:
# Split X (the features) and y (the labels) into traing and testing sets:
# X_train, y_train, X_test, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=47)
y_train=y_train.T 
y_test=y_test.T
#Printing the splitted data for testing and training the model
print("X_train data is: ",X_train)
print("y_train data is: ",y_train)
print("X_test data is: ",X_test)
print("y_test data is: ",y_test)



In [ ]:
# The algorithm we discussed in class assumes that the features array to have the dimensions n rows by m columns
# where n is the number of features and m is the size of the data
# Make sure that X_train and X_test have the correct dimensions
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


In [ ]:
# Define the sigmoid function and the derivative of the sigmoid function

#Activation function sigmoid is defined below\

def g1(x):
    return 1. / (1 + np.exp(-x))
def g2(x):
    return 1. / (1 + np.exp(-x))
def g1_prime(x):
    return g1(x) * (1 - g1(x))


In [ ]:
# Define a function to initialize the weight and bias matrices

def intialize(n_h,n_x,n_y):
    W1 = np.random.randn(n_h, n_x) * 0.1
    b1 = np.random.randn(n_h, 1)
    W2 = np.random.randn(n_y, n_h)
    b2 = np.random.randn(n_y, 1)
    
    return W1,b1,W2,b2

In [ ]:
# Include the function "nn" that does the training of a model
def NN(X, y, n_h, n_y, alpha, iterations):
    stage_1 = iterations//2; alpha2 = alpha/2.
    stage_2 = iterations//1.75; alpha3 = alpha/4.
    stage_3 = iterations//1.5; alpha4 = alpha/8.
    cost_list = [[],[]]
    m = X.shape[1]
    n_x = X.shape[0]
    
    #Calling the above declared intialize method to set the weight and bias matrices 
    W1,b1,W2,b2=intialize(n_h,n_x,n_y)
   
    for i in range(iterations):
        # Forward propagation 
        Z1 = np.dot(W1, X) + b1
        A1 = g1(Z1)
        Z2 = np.dot(W2, A1) + b2
        A2 = g2(Z2)
        
        #Loss function
        if i%100 == 0:
            # Usage of Cross Entropy to calculate loss
            cost = -np.sum(y*np.log(A2) + (1-y)*np.log(1-A2))
            # Using root mean square method to calculate loss
            #cost = np.sum(np.sqrt(((y-A2)**2).mean()))
            cost_list[0].append(i)
            cost_list[1].append(cost)
        #Backward propagation  
        dZ2 = A2 - y
        dW2 = (1/m) * np.dot(dZ2, A1.T)
        db2 = (1/m) * np.sum(dZ2)
        dZ1 = np.dot(W2.T, dZ2) * g1_prime(Z1)
        dW1 = (1/m) * np.dot(dZ1, X.T)
        db1 = (1/m) * np.sum(dZ1)
        W2 = W2 - alpha * dW2
        b2 = b2 - alpha * db2
        W1 = W1 - alpha * dW1
        b1 = b1 - alpha * db1
        if i>stage_1: alpha = alpha2
        elif i>stage_2: alpha = alpha3
        elif i>stage_3: alpha = alpha4
    return W1, b1, W2, b2, cost_list

In [ ]:
# Define the parameters needed for traing the model
# Initialize your weight matrices
# call the the "nn" function to train the model on the testing data

#Intializing the input data to train the model
X = np.array(X_train).T 
#Intializing the expected output data to feed the model
y = np.array(y_train)
n_h = 8; n_y = 1

#Alpha is the learning rate 
alpha = 0.05
iterations = 50000

#Calling the NN function to train the model by providing the required parameters 
W1, b1, W2, b2, cost_list = NN(X, y, n_h, n_y, alpha, iterations)

#Printing the final adjusted parameters to train the model
print('W1=',W1, '\n', 'b1=',b1, '\n', 'W2=',W2, '\n', 'b2=',b2)


In [ ]:
# Show a plot of the loss as a function of the number of iterations

plt.plot(cost_list[0][1:], cost_list[1][1:], 'go')


In [ ]:
# Using the trained model, predict the labels for the testing data

#Testing the model with the test data 

Z1 = np.dot(W1, X_test.T) + b1
A1 = g1(Z1)
Z2 = np.dot(W2, A1) + b2
A2 = g2(Z2)
predictions = A2.copy()
predictions[A2 < 0.5] = 0
predictions[A2 > 0.5] = 1
print('A2=',A2)
print('predictions=',predictions)

In [ ]:
# Make sure that the array of the predicted labels has the same dimension as the testing data

print("Shape of testing data:", y_test.shape)
print("Shape of predictions data: ",predictions.shape)


In [ ]:
# Show the confusion matrix for the testing data
from sklearn.metrics import classification_report, confusion_matrix 

print(confusion_matrix(y_test[0], predictions[0]))  
print(classification_report(y_test[0], predictions[0])) 


Step 3:
    
Below is the confusion matrix and classification report when the model is trained with 80 perecent training data and tested with 20 percent testing data and the number of neurons are 8 and iterations are 50000 here.

[[84  1]
 [ 3 52]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98        85
           1       0.98      0.95      0.96        55

    accuracy                           0.97       140
   macro avg       0.97      0.97      0.97       140
weighted avg       0.97      0.97      0.97       140

Step 4:

Below is the confusion matrix and classification report when the model is trained the 5 neurons in the hidden layer
[[84  1]
 [ 4 51]]
              precision    recall  f1-score   support

           0       0.95      0.99      0.97        85
           1       0.98      0.93      0.95        55

    accuracy                           0.96       140
   macro avg       0.97      0.96      0.96       140
weighted avg       0.96      0.96      0.96       140


Below is the confusion matrix and classification report when the model is trained the 10 neurons in the hidden layer

[[84  1]
 [ 4 51]]
              precision    recall  f1-score   support

           0       0.95      0.99      0.97        85
           1       0.98      0.93      0.95        55

    accuracy                           0.96       140
   macro avg       0.97      0.96      0.96       140
weighted avg       0.96      0.96      0.96       140


Step 5:

Below is the confusion matrix and classification report when the model is trained with th 90 perecent training data and tested with 10 percent testing data

[[44  1]
 [ 2 23]]
              precision    recall  f1-score   support

           0       0.96      0.98      0.97        45
           1       0.96      0.92      0.94        25

    accuracy                           0.96        70
   macro avg       0.96      0.95      0.95        70
weighted avg       0.96      0.96      0.96        70

Below is the confusion matrix and classification report when the model is trained with th 60 perecent training data and tested with 40 percent testing data

[[171  10]
 [  7  92]]
              precision    recall  f1-score   support

           0       0.96      0.94      0.95       181
           1       0.90      0.93      0.92        99

    accuracy                           0.94       280
   macro avg       0.93      0.94      0.93       280
weighted avg       0.94      0.94      0.94       280


Conclusion: Accuracy is a bit high when the model is feed with 90 percent of the data for training the model compared to the 60 percent 


Step 6:

Below is the confusion matrix, classification report and acurracy of model when we use the cross entropy as the loss function  


[[84  1]
 [ 3 52]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98        85
           1       0.98      0.95      0.96        55

    accuracy                           0.97       140
   macro avg       0.97      0.97      0.97       140
weighted avg       0.97      0.97      0.97       140


Below is the confusion matrix, classification report and acurracy of model when we use the Root Mean square as the loss function 

[[84  1]
 [ 5 50]]
              precision    recall  f1-score   support

           0       0.94      0.99      0.97        85
           1       0.98      0.91      0.94        55

    accuracy                           0.96       140
   macro avg       0.96      0.95      0.95       140
weighted avg       0.96      0.96      0.96       140

# Using batches to train the model

In [ ]:
def NN_batchModel(X, y, n_h, n_y, alpha, iterations):
    stage_1 = iterations//2; alpha2 = alpha/2.
    stage_2 = iterations//1.75; alpha3 = alpha/4.
    stage_3 = iterations//1.5; alpha4 = alpha/8.
    cost_list = [[],[]]
    m = X.shape[1]
    n_x = X.shape[0]
    batch_size=64
    W1,b1,W2,b2=intialize(n_h,n_x,n_y)
    
    for i in range(iterations):
        for j in range(0, m, batch_size):
            X_batch = X[:, j:j+batch_size]
            y_batch = y[j:j+batch_size]
            Z1 = np.dot(W1, X_batch) + b1
            A1 = g1(Z1)
            Z2 = np.dot(W2, A1) + b2
            A2 = g1(Z2)
            dZ2 = A2 - y_batch
            dW2 = (1/m) * np.dot(dZ2, A1.T)
            db2 = (1/m) * np.sum(dZ2)
            dZ1 = np.dot(W2.T, dZ2) * g1_prime(Z1)
            dW1 = (1/m) * np.dot(dZ1, X_batch.T)
            db1 = (1/m) * np.sum(dZ1)
            W2 = W2 - alpha * dW2
            b2 = b2 - alpha * db2
            W1 = W1 - alpha * dW1
            b1 = b1 - alpha * db1
            if i>stage_3: alpha = alpha4
            elif i>stage_2: alpha = alpha3
            elif i>stage_1: alpha = alpha2
                
        if i%2 == 0:
            Z1 = np.dot(W1, X) + b1
            A1 = g1(Z1)
            Z2 = np.dot(W2, A1) + b2
            A2 = g1(Z2)
            cost = -np.sum(y*np.log(A2) + (1-y)*np.log(1-A2))
            cost_list[0].append(i)
            cost_list[1].append(cost)
    return W1, b1, W2, b2, cost_list

In [ ]:
X = (X_train).T 
#Intializing the expected output data to feed the model
y = np.ravel((y_train))

n_h = 8; n_y = 1

#Alpha is the learning rate 
alpha = 0.05
iterations = 10000

#Calling the NN function to train the model by providing the required parameters 
W1, b1, W2, b2, cost_list = NN_batchModel(X, y, n_h, n_y, alpha, iterations)

#Printing the final adjusted parameters to train the model
print('W1=',W1, '\n', 'b1=',b1, '\n', 'W2=',W2, '\n', 'b2=',b2)


In [ ]:
# Plot of the loss as a function of the number of iterations when using batches to train the model

plt.plot(cost_list[0][1:], cost_list[1][1:], 'go')

In [ ]:
#Testing the model after it being trained using the batches 

Z1 = np.dot(W1, X_test.T) + b1
A1 = g1(Z1)
Z2 = np.dot(W2, A1) + b2
A2 = g2(Z2)
predictions = A2.copy()
predictions[A2 < 0.5] = 0
predictions[A2 > 0.5] = 1
print('predictions=',predictions)

from sklearn.metrics import classification_report, confusion_matrix 

print(confusion_matrix(y_test[0], predictions[0]))  
print(classification_report(y_test[0], predictions[0])) 

Step 7

Below is the confusion matrix and classification report when the batch size is used as 32 and number of iterations used here are less compared to the normal training model i.e 10000 iterations

[[84  1]
 [ 3 52]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98        85
           1       0.98      0.95      0.96        55

    accuracy                           0.97       140
   macro avg       0.97      0.97      0.97       140
weighted avg       0.97      0.97      0.97       140


Below is the confusion matrix and classification report when the batch size is used as 64

[[84  1]
 [ 4 51]]
              precision    recall  f1-score   support

           0       0.95      0.99      0.97        85
           1       0.98      0.93      0.95        55

    accuracy                           0.96       140
   macro avg       0.97      0.96      0.96       140
weighted avg       0.96      0.96      0.96       140


Conclusion: When the batch size is increased the training time is decreased but the accuracy of the model is reduced, so the smaller batch sizes gives more accuracy compared to the larger batch size
